# Setup

In [1]:
!nvidia-sml

/usr/bin/bash: línea 1: nvidia-sml: orden no encontrada


In [2]:
!uv pip install ultralytics
import ultralytics
ultralytics.checks()

Ultralytics 8.3.211 🚀 Python-3.9.23 torch-2.8.0+cu128 CPU (Intel Core i5-1035G1 1.00GHz)
Setup complete ✅ (8 CPUs, 7.6 GB RAM, 30.0/50.9 GB disk)


https://docs.ultralytics.com/es/modes/predict/#key-features-of-predict-mode

Validar la precision con los splits `val` y `test`.
*Validate YOLO11n on COCO8 val:*   
!yolo val model=yolo11n.pt data=coco8.yaml

# Train

In [7]:
#Select YOLO11 logger {run: 'auto'}
logger = 'TensorBoard' #@param ['TensorBoard', 'Weights & Biases']

if logger == 'TensorBoard':
  !yolo settings tensorboard=True
  %load_ext tensorboard
  %tensorboard --logdir .
elif logger == 'Weights & Biases':
  !yolo settings wandb=True

✅ Updated 'tensorboard=True'
JSONDict("/home/valentin/.config/Ultralytics/settings.json"):
{
  "settings_version": "0.0.6",
  "datasets_dir": "/home/valentin/Universidad/tercer-anio/Proyecto/datasets",
  "weights_dir": "/home/valentin/Universidad/tercer-anio/Proyecto/proyecto-2025/weights",
  "runs_dir": "/home/valentin/Universidad/tercer-anio/Proyecto/proyecto-2025/runs",
  "uuid": "68fcec538c6e52face0c9e9ba17ef73499468cbf851d2f47dcf4db2f18089b9f",
  "sync": true,
  "api_key": "",
  "openai_api_key": "",
  "clearml": true,
  "comet": true,
  "dvc": true,
  "hub": true,
  "mlflow": true,
  "neptune": true,
  "raytune": true,
  "tensorboard": true,
  "wandb": false,
  "vscode_msg": true,
  "openvino_msg": false
}
💡 Learn more about Ultralytics Settings at https://docs.ultralytics.com/quickstart/#ultralytics-settings


## Entrenar con Python.
[Documentacion aca.](https://docs.ultralytics.com/es/usage/python/)

In [5]:
from ultralytics import YOLO

# Load a model
model = YOLO('yolo11s.yaml')  # build a new model from scratch
model = YOLO('yolo11s.pt')  # load a pretrained model (recommended for training)

# Use the model
results = model.train(data='coco8.yaml', epochs=3)  # train the model
results = model.val()  # evaluate model performance on the validation set
results = model('https://ultralytics.com/images/bus.jpg')  # predict on an image
results = model.export(format='onnx')  # export the model to ONNX format

New https://pypi.org/project/ultralytics/8.3.212 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.211 🚀 Python-3.9.23 torch-2.8.0+cu128 CPU (Intel Core i5-1035G1 1.00GHz)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=coco8.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=3, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train2, nbs=64, nms=False, opset=None, optimize=Fa

# Predict
Se utiliza para realizar predicciones utilizando un modelo YOLO entrenado en nuevas imágenes o videos. El modelo se carga desde un archivo de punto de control.

Al ultimo tiene una funcion dada por chatGPT que exporta las salidas de la inferencia como un archivo json

In [ ]:
from ultralytics import YOLO
import json
model = YOLO('yolo11s.pt')

results = model.predict(source='./bus.jpg', show=True)

results[0].show()

# Exportar el modelo
Exportar el modelo para utilizarlo en el captioning

In [ ]:
model.export(format="pt")

## Extraer features
Bloque para extraer las features del modelo, que despues se van a usar para entrenar el captioning.

In [ ]:
# =====================================
# EXTRAER FEATURES CON YOLO11
# (Con ayuda de chatGPT jaj)
# =====================================
import os
import torch
import numpy as np
from ultralytics import YOLO
from PIL import Image

# 1. Configuración
#IMAGES_DIR es la carpeta de las imagenes CRUDAS (sin pre processing) del dataset
IMAGES_DIR = "./datasets/flickr8r/images/"
#OUTPUT_DIR es la carpeta a la que se van a guardar los vectores .npy que contienen las
#  features de cada imagen
OUTPUT_DIR = "r"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# 2. Cargar modelo YOLO
model = YOLO("yolo11s.pt")
backbone = model.model.model[:10]  # primeras capas del backbone

# 3. Función auxiliar para extraer vector
def extract_features_yolo(image_path):
    img = Image.open(image_path).convert("RGB")
    tensor = model.preprocess(img)  # preprocesado interno de YOLO
    with torch.no_grad():
        features = backbone(tensor)  # (1, C, H', W')
        vector = torch.mean(features, dim=(2, 3))  # (1, C)
    return vector.squeeze(0).cpu().numpy()

# 4. Procesar todas las imágenes y guardar
for img_name in os.listdir(IMAGES_DIR):
    if not img_name.lower().endswith(('.jpg', '.jpeg', '.png')):
        continue
    path = os.path.join(IMAGES_DIR, img_name)
    feats = extract_features_yolo(path)
    np.save(os.path.join(OUTPUT_DIR, img_name.split('.')[0] + ".npy"), feats)

print(f"✅ Features guardadas en {OUTPUT_DIR}")
